In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
! pip install transformers -q
! pip install tokenizers -q

In [5]:
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
import numpy as np
import pandas as pd
from typing import *
from tqdm.notebook import tqdm
from sklearn.utils.extmath import softmax
from sklearn import model_selection
from sklearn.metrics import classification_report, f1_score, roc_auc_score

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import AdamW
import tokenizers

In [7]:
def seed_all(seed = 42):
  """
  Fix seed for reproducibility
  """
  # python RNG
  import random
  random.seed(seed)

  # pytorch RNGs
  import torch
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

  # numpy RNG
  import numpy as np
  np.random.seed(seed)

In [8]:
import os
os.chdir('/gdrive/My Drive/Sentihood/')

In [9]:
class config:
  SEED = 42
  KFOLD = 5
  TRAIN_FILE = 'train_multi.csv'
  VAL_FILE = 'dev_multi.csv'
  TEST_FILE = 'test_multi.csv'
  PSEUDO_FILE = os.path.join('run_bert_multi_20', 'pseudo_labels.csv')
  COMPLETE_TEST = 'complete_test.csv'
  SAVE_DIR = 'run_bert_multi_pseudo'
  MAX_LEN = 192
  MODEL = './bert-base-uncased'
  TOKENIZER = transformers.BertTokenizer.from_pretrained('./bert-base-uncased')
  EPOCHS = 7 #5*2
  TRAIN_BATCH_SIZE = 16
  VALID_BATCH_SIZE = 16

In [10]:
# save pretrained weights
# m = transformers.BertModel.from_pretrained('bert-base-uncased')
# t = transformers.BertTokenizer.from_pretrained('./bert-base-uncased')
# ! mkdir bert-base-uncased
# m.save_pretrained('./bert-base-uncased')
# t.save_vocabulary('./bert-base-uncased')

In [11]:
class AverageMeter:
    """
    Computes and stores the average value
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
class EarlyStopping:
    """
    Early stopping utility
    Source : https://www.kaggle.com/abhishek/bert-base-uncased-using-pytorch/
    """
    
    def __init__(self, patience=7, mode="max", delta=0.001):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):
        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            print('Validation score improved ({} --> {}). Saving model!'.format(self.val_score, epoch_score))
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [13]:
df = pd.read_csv(config.TRAIN_FILE)
list_aspects = []
for i, r in df.iterrows():
  list_aspects.extend(r['aspect'].split(' '))

aspects = sorted(set(list_aspects))

aspect2idx = {v:k for k,v in enumerate(aspects)}
idx2aspect = {v:k for k,v in aspect2idx.items()}

In [14]:
sentiment2idx = {'Negative':1, 'Positive':2, 'None':0}
idx2sentiment = {0:'None', 1:'Negative', 2:'Positive'}

In [15]:
def process_data(text, target, aspect, sentiment, tokenizer, max_len):

  text = str(text).strip()
  target = str(target).strip()
 
  question = f'find about {target}'

  tokens = tokenizer(text = question, text_pair=text)

  aspect = aspect.split(' ')
  sentiment = sentiment.split(' ')

  aspect_class = [0]*len(aspect2idx)
  for i in aspect:
    aspect_class[aspect2idx[i]] = 1

  sentiment_class = [0]*len(aspect2idx)
  for i, j in zip(aspect, sentiment):
    sentiment_class[aspect2idx[i]] = sentiment2idx[j]


  token_ids = tokens.input_ids
  token_type = tokens.token_type_ids
  mask = [1] * len(token_ids)  

  padding = max_len - len(token_ids)
  
  if padding>=0:
    token_ids = token_ids + ([0] * padding)
    mask = mask + ([0] * padding)
    token_type = token_type + [1] * padding
    
  else:
    token_ids = token_ids[0:max_len]
    mask = mask[0:max_len]
    token_type = token_type[0:max_len]
    
  
  assert len(token_ids)==max_len
  assert len(mask)==max_len
  assert len(token_type)==max_len

  
  return {'text': text,
          'ids': token_ids,
          'token_type': token_type,
          'mask': mask,
          'aspect': aspect_class,
          'sentiment': sentiment_class,
        }

In [16]:
class Dataset:
    def __init__(self, text, target, aspect, sentiment):
        self.text = text
        self.target = target
        self.aspect = aspect
        self.sentiment = sentiment
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        data = process_data(
            self.text[item],
            self.target[item],
            self.aspect[item], 
            self.sentiment[item],
            self.tokenizer,
            self.max_len,
        )

        return {
            'ids': torch.tensor(data["ids"], dtype=torch.long),
            'mask': torch.tensor(data["mask"], dtype=torch.long),
            'token_type': torch.tensor(data['token_type'], dtype=torch.long),
            'aspect': torch.tensor(data['aspect'], dtype=torch.float32),
            'sentiment': torch.tensor(data['sentiment'], dtype=torch.long),
        }

In [17]:
def get_loss(aspect, aspect_logits, sentiment, sentiment_logits):
  loss_fn1 = nn.BCEWithLogitsLoss()
  loss_fn2 = nn.CrossEntropyLoss()

  aspect_loss = loss_fn1(aspect_logits, aspect)
  
  # adjusting dimensionality for CrossEntropy loss
  sentiment_logits = sentiment_logits.view(-1, len(sentiment2idx))
  sentiment = sentiment.view(-1)

  sentiment_loss = loss_fn2(sentiment_logits, sentiment)

  loss = aspect_loss + sentiment_loss
  return loss


In [18]:
class BertM(nn.Module):
  def __init__(self):
    super(BertM, self).__init__()
    self.bert = transformers.BertModel.from_pretrained(config.MODEL, output_hidden_states=False)
    self.aspect_cls =  nn.Linear(768, len(aspect2idx))
    self.sentiment_cls = nn.Linear(768, len(sentiment2idx)*len(aspect2idx))
    self.drop = nn.Dropout(0.1)
    

  def forward(self, ids, mask, token_type, aspect=None, sentiment=None):
    
    output = self.bert(input_ids = ids, attention_mask = mask, token_type_ids=token_type)
    out = self.drop(output[1])

    aspect_logits = self.aspect_cls(out)
    sentiment_logits = self.sentiment_cls(out).view(-1, len(aspect2idx), len(sentiment2idx)) 

    loss = get_loss(aspect, aspect_logits, sentiment, sentiment_logits)    

    return loss, aspect_logits, sentiment_logits

In [19]:
def train_fn(data_loader, model, optimizer, device):
  model.train()
  losses = AverageMeter()
  tk0 = tqdm(data_loader, total=len(data_loader))
  
  for bi, d in enumerate(tk0):
    
    ids = d['ids'].to(device, dtype=torch.long)
    token_type = d['token_type'].to(device, dtype=torch.long)
    mask = d['mask'].to(device, dtype=torch.long)
    aspect = d['aspect'].to(device, dtype=torch.float32)
    sentiment = d['sentiment'].to(device, dtype=torch.long)
    

    model.zero_grad()
    loss, aspect_logits, sentiment_logits = model(ids, mask, token_type, aspect, sentiment)

    loss.backward()
    optimizer.step()

    losses.update(loss.item(), ids.size(0))
    tk0.set_postfix(loss=losses.avg)


In [20]:
def one_hot(a, num_classes):
  return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax_custom(x):
  return np.exp(x)/np.sum(np.exp(x), axis=-1)

In [21]:
def eval_fn(data_loader, model, device):
  model.eval()
  losses = AverageMeter()
  
  tk0 = tqdm(data_loader, total=len(data_loader))
  
  a_yt, a_ypb, a_yp = [], [], []
  s_yt, s_ypb, s_yp = [], [], []
  
  
  for bi, d in enumerate(tk0):
    ids = d['ids'].to(device, dtype=torch.long)
    token_type = d['token_type'].to(device, dtype=torch.long)
    mask = d['mask'].to(device, dtype=torch.long)
    aspect = d['aspect'].to(device, dtype=torch.float32)
    sentiment = d['sentiment'].to(device, dtype=torch.long)
    
    with torch.no_grad():
      loss, aspect_logits, sentiment_logits = model(ids, mask, token_type, aspect, sentiment)
  

    aspect_logits = aspect_logits.detach().cpu().numpy()
    sentiment_logits = sentiment_logits.detach().cpu().numpy()

    aspect_prob = sigmoid(aspect_logits)

    sentiment_prob = np.zeros(sentiment_logits.shape)

    for i in range(sentiment_prob.shape[0]):
      sentiment_prob[i] = softmax(sentiment_logits[i])
    
    aspect_pred = np.where(aspect_prob>=0.5, 1, 0)  
    sentiment_pred = np.argmax(sentiment_prob, axis=-1)

    aspect_truth = d['aspect'].to('cpu').numpy()
    sentiment_truth = d['sentiment'].to('cpu').numpy()

    aspect_truth = aspect_truth.flatten()
    sentiment_truth = sentiment_truth.flatten()

    aspect_prob = aspect_prob.flatten()
    sentiment_prob = sentiment_prob.reshape(-1, len(sentiment2idx))
    
    aspect_pred = aspect_pred.flatten()
    sentiment_pred = sentiment_pred.flatten()
    

    a_yt = a_yt + aspect_truth.tolist()
    a_ypb = a_ypb + aspect_prob.tolist()
    a_yp = a_yp + aspect_pred.tolist()

    s_yt = s_yt + sentiment_truth.tolist()
    s_ypb = s_ypb + sentiment_prob.tolist()
    s_yp = s_yp + sentiment_pred.tolist()
      
    losses.update(loss.item(), ids.size(0))

    tk0.set_postfix(loss=losses.avg)

  a_yt = np.array(a_yt)
  s_yt = np.array(s_yt)

  a_yp = np.array(a_yp)
  s_yp = np.array(s_yp)

  a_ypb = np.vstack(a_ypb).flatten()
  s_ypb = np.vstack(s_ypb).reshape(-1, len(sentiment2idx))
  

  print(f'Aspect : F1 score = {f1_score(a_yt, a_yp, average="macro")} | AUC-ROC = {roc_auc_score(a_yt, a_ypb, average="macro")} ')
  print(f'Sentiment : F1 score = {f1_score(s_yt, s_yp, average="macro")} | AUC-ROC = {roc_auc_score(one_hot(s_yt, len(sentiment2idx)), s_ypb, average="macro")} ')
  
  return losses.avg 
  

In [22]:
def run(df_train, df_val, fold=None):
  train_dataset = Dataset(
        text = df_train.text.values,
        target = df_train.target.values,
        aspect = df_train.aspect.values,
        sentiment = df_train.sentiment.values,
    )
  
  valid_dataset = Dataset(
        text = df_val.text.values,
        target = df_val.target.values,
        aspect = df_val.aspect.values,
        sentiment = df_val.sentiment.values,
    )
  
    
  train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=4
    )

  valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )

  model = BertM()
  device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
  model.to(device)
  # torch.save(model.state_dict(), os.path.join(config.SAVE_DIR, f"model_{fold}.bin"))

  lr = 5e-5
  param_optimizer = list(model.named_parameters())
  no_decay = ['bias', 'gamma', 'beta']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.01},
      {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
      'weight_decay_rate': 0.0}
  ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

  es = EarlyStopping(patience=3, mode="min")

  print('Starting training....')
  for epoch in range(config.EPOCHS):
    train_fn(train_data_loader, model, optimizer, device)
    valid_loss = eval_fn(valid_data_loader, model, device)
    print(f'Epoch : {epoch + 1} | Validation Score :{valid_loss}')
    if fold is None:
      es(valid_loss, model, model_path=os.path.join(config.SAVE_DIR, "model.bin"))
    else:
      es(valid_loss, model, model_path=os.path.join(config.SAVE_DIR, f"model_{fold}.bin"))
    if es.early_stop:
      print("Early stopping")
      break

  return

In [23]:
def run_k_fold():
  '''
    Perform k-fold cross-validation
  '''

  seed_all()
  
  df_train = pd.read_csv(config.TRAIN_FILE)
  df_val = pd.read_csv(config.VAL_FILE)
  
  df_extra = pd.read_csv(config.PSEUDO_FILE)
  df_extra.dropna(inplace=True)
  
  df_pseudo = pd.DataFrame({'id': df_extra['id'].values,
                            'text': df_extra['text'].values,
                            'aspect': df_extra['pred_aspect'].values,
                            'sentiment': df_extra['pred_sentiment'].values,
                            })


  # concatenating train and validation set
  train = pd.concat([df_train, df_val, df_pseudo]).reset_index()
  

  # dividing folds
  kf = model_selection.StratifiedKFold(n_splits=config.KFOLD, shuffle=False, random_state=config.SEED)
  for fold, (train_idx, val_idx) in enumerate(kf.split(X=train, y=train.aspect.values)):
      train.loc[val_idx, 'kfold'] = fold

  
  aspect, sentiment = [], []

  for i in range(config.KFOLD):
    print(f'################################################ Fold {i} ##################################################')
    df_train = train[train.kfold!=i]
    df_val = train[train.kfold==i]
    run(df_train, df_val, i)
  

In [24]:
def test_fn(data_loader, model, device, cal_metric=True):
  model.eval()
  losses = AverageMeter()
  
  tk0 = tqdm(data_loader, total=len(data_loader))
  
  a_yt, a_ypb, a_yp = [], [], []
  s_yt, s_ypb, s_yp = [], [], []
  
  for bi, d in enumerate(tk0):
    ids = d['ids'].to(device, dtype=torch.long)
    token_type = d['token_type'].to(device, dtype=torch.long)
    mask = d['mask'].to(device, dtype=torch.long)
    aspect = d['aspect'].to(device, dtype=torch.float32)
    sentiment = d['sentiment'].to(device, dtype=torch.long)
    
    with torch.no_grad():
      loss, aspect_logits, sentiment_logits = model(ids, mask, token_type, aspect, sentiment)
  

    aspect_logits = aspect_logits.detach().cpu().numpy()
    sentiment_logits = sentiment_logits.detach().cpu().numpy()

    aspect_prob = sigmoid(aspect_logits) # (bs, a)
    sentiment_prob = np.zeros(sentiment_logits.shape) # (bs, a, s) 

    
    for i in range(sentiment_prob.shape[0]):
      sentiment_prob[i] = softmax(sentiment_logits[i])
    
    aspect_pred = np.where(aspect_prob>=0.5, 1, 0)  # (bs, a)
    sentiment_pred = np.argmax(sentiment_prob, axis=-1) # (bs, a)


    aspect_truth = d['aspect'].to('cpu').numpy() # (bs, a)
    sentiment_truth = d['sentiment'].to('cpu').numpy() # (bs, a)

    aspect_truth = aspect_truth.flatten()
    sentiment_truth = sentiment_truth.flatten()

    aspect_prob = aspect_prob.flatten()
    sentiment_prob = sentiment_prob.reshape(-1, len(sentiment2idx))
    
    aspect_pred = aspect_pred.flatten()
    sentiment_pred = sentiment_pred.flatten()
    

    a_yt = a_yt + aspect_truth.tolist()
    a_ypb = a_ypb + aspect_prob.tolist()
    a_yp = a_yp + aspect_pred.tolist()

    s_yt = s_yt + sentiment_truth.tolist()
    s_ypb = s_ypb + sentiment_prob.tolist()
    s_yp = s_yp + sentiment_pred.tolist()
      
    losses.update(loss.item(), ids.size(0))

    tk0.set_postfix(loss=losses.avg)
    

  a_yt = np.array(a_yt)
  s_yt = np.array(s_yt)

  a_yp = np.array(a_yp)
  s_yp = np.array(s_yp)

  a_ypb = np.vstack(a_ypb).flatten()
  s_ypb = np.vstack(s_ypb).reshape(-1, len(sentiment2idx))
  
  if cal_metric:
    print(f'Aspect : F1 score = {f1_score(a_yt, a_yp, average="macro")} | AUC-ROC = {roc_auc_score(a_yt, a_ypb, average="macro")} ')
    print(f'Sentiment : F1 score = {f1_score(s_yt, s_yp, average="macro")} | AUC-ROC = {roc_auc_score(one_hot(s_yt, len(sentiment2idx)), s_ypb, average="macro")} ')
  
  # changing to original shape
  a_ypb = a_ypb.reshape(-1, len(aspect2idx))
  s_ypb = s_ypb.reshape(-1, len(aspect2idx), len(sentiment2idx))

  return a_ypb, s_ypb
  

In [25]:
def generate_results(df_test, cal_metric=False):
  test_dataset = Dataset(
        text = df_test.text.values,
        target = df_test.target.values,
        aspect = df_test.aspect.values,
        sentiment = df_test.sentiment.values,
    )
  test_data_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=2
    )

  model = BertM()
  sentiment, aspect = [], []
  device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

  for fold in range(config.KFOLD):
    model.load_state_dict(torch.load(os.path.join(config.SAVE_DIR, f"model_{fold}.bin")))
    model.to(device)
    ap, sp = test_fn(test_data_loader, model, device, cal_metric)
    aspect.append(ap)
    sentiment.append(sp)

  # avg probabilities
  avg_aspect = sum(aspect)/config.KFOLD # (bs, a)
  avg_sentiment = sum(sentiment)/config.KFOLD # (bs, a, s)

  aspect_pred = np.where(avg_aspect>=0.5, 1, 0)  # (bs, a)
  sentiment_pred = np.argmax(avg_sentiment, axis=-1) # (bs, a)

  predicted_aspect, predicted_sentiment = [], []
    
  for i in range(aspect_pred.shape[0]):
    a, s = [], []
    for j in range(aspect_pred.shape[1]):
      ## aspect is present and sentiment is not "None"
      if aspect_pred[i][j]==1 and sentiment_pred[i][j]!=0:
        a.append(idx2aspect[j])
        s.append(idx2sentiment[sentiment_pred[i][j]])
    predicted_aspect.append(' '.join(a))
    predicted_sentiment.append(' '.join(s))

  df_test['pred_aspect'] = predicted_aspect
  df_test['pred_sentiment'] = predicted_sentiment

  a_yt, s_yt = [], []

  for d in test_data_loader:
    a_yt.append(d['aspect'])
    s_yt.append(d['sentiment'])

  a_yt = np.vstack(a_yt).flatten()
  s_yt = np.vstack(s_yt).flatten()

  avg_aspect = avg_aspect.reshape(-1)
  avg_sentiment = avg_sentiment.reshape(-1, len(sentiment2idx))

  aspect_pred = aspect_pred.flatten()
  sentiment_pred = sentiment_pred.flatten()
  
  if cal_metric:
    print('Final Results : ')
    print(f'Aspect : F1 score = {f1_score(a_yt, aspect_pred, average="macro")}\
    | AUC-ROC = {roc_auc_score(a_yt, avg_aspect, average="macro")} ')
    
    print(f'Sentiment : F1 score = {f1_score(s_yt, sentiment_pred, average="macro")}\
    | AUC-ROC = {roc_auc_score(one_hot(s_yt, len(sentiment2idx)), avg_sentiment, average="macro")} ')


  return df_test

In [26]:
if __name__=='__main__':
  ! rm -rf {config.SAVE_DIR} && mkdir {config.SAVE_DIR}
  print('Starting training....')
  run_k_fold()
  
  print('Predicting for labeled test dataset....')
  df_test = pd.read_csv(config.TEST_FILE)
  df_test = generate_results(df_test, True)
  df_test.to_csv(os.path.join(config.SAVE_DIR, 'sub.csv'), index=False)

  print('Generating predictions for complete test data....')
  pseudo = pd.read_csv(config.COMPLETE_TEST)
  df = generate_results(pseudo, cal_metric=False)
  df.to_csv(os.path.join(config.SAVE_DIR, 'predictions.csv'), index=False)


Starting training....


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


################################################ Fold 0 ##################################################
Starting training....



Aspect : F1 score = 0.7887559737156511 | AUC-ROC = 0.9398267524097268 
Sentiment : F1 score = 0.6088101095233008 | AUC-ROC = 0.9287072080602083 
Epoch : 1 | Validation Score :0.41942314006643283
Validation score improved (inf --> 0.41942314006643283). Saving model!



Aspect : F1 score = 0.8866333257741903 | AUC-ROC = 0.9720354999746461 
Sentiment : F1 score = 0.672966886045407 | AUC-ROC = 0.9617314522502097 
Epoch : 2 | Validation Score :0.28528834191517116
Validation score improved (0.41942314006643283 --> 0.28528834191517116). Saving model!


...


Aspect : F1 score = 0.9225600993277274 | AUC-ROC = 0.9834661647163296 
Sentiment : F1 score = 0.8254415038863495 | AUC-ROC = 0.9794422482281157 



Aspect : F1 score = 0.9204124530760076 | AUC-ROC = 0.9772991786903041 
Sentiment : F1 score = 0.8303451020812241 | AUC-ROC = 0.97642386806838 



Aspect : F1 score = 0.9280621574881947 | AUC-ROC = 0.9785865883508625 
Sentiment : F1 score = 0.8391271342358091 | AUC-ROC = 0.9752260803757195 



Aspect : F1 score = 0.9215769311310935 | AUC-ROC = 0.9835386776785294 
Sentiment : F1 score = 0.8213162936369082 | AUC-ROC = 0.9805435953008628 



Aspect : F1 score = 0.9271714660644268 | AUC-ROC = 0.9812978601907674 
Sentiment : F1 score = 0.8398381938205336 | AUC-ROC = 0.9780751115190229 
Final Results : 
Aspect : F1 score = 0.9310735090352005    | AUC-ROC = 0.9881326693907534 
Sentiment : F1 score = 0.8463187241292686    | AUC-ROC = 0.9859811724966535 
Generating predictions for complete test data....
